# Fetch Live Weather Data (For Implementation in App)

This code is used to fetch the weather forecast for usage in the streamlit app.

In [24]:
import requests
from datetime import date, datetime, timedelta
import pytz
import pandas as pd

In [ ]:
# Install the package to fetch energy demand data
import sys
!{sys.executable} -m pip install git+https://github.com/tomharvey/ElexonDataPortal.git@pandas-version-tz-compare

In [17]:
# API package for energy demand data
from ElexonDataPortal import api

In [18]:
# API keys
from secret import api_key
weather_key = '3c1e7c92b954a643255bf4c79684fddb'

## Fetch Live Demand Data

In [19]:
client = api.Client(api_key)

In [20]:
time = datetime.utcnow()
demand_df = client.get_SYSDEM((time - timedelta(days=28)).strftime("%Y-%m-%d"), (time + timedelta(days=28)).strftime("%Y-%m-%d"))

/opt/homebrew/lib/python3.11/site-packages/ElexonDataPortal/dev/utils.py:119: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  df_dates_SPs = df_dates_SPs[start_date:end_date]


In [21]:
last_demand = demand_df.loc[demand_df['recordType'] == 'ITSDO'].iloc[-1]

## Fetch Live Weather Data

In [22]:
locations = {
    'london': {
        'lat': 51.503655,
        'lon': -0.105732
    },
    'liverpool': {
        'lat': 53.402859,
        'lon': -2.969904
    },
    'bath': {
        'lat': 51.387535,
        'lon': -2.373896
    }
}

In [25]:
weather_dfs = []
for key, loc in locations.items():
    url = f"https://api.openweathermap.org/data/2.5/forecast?lat={loc['lat']}&lon={loc['lon']}&appid={weather_key}"
    response = requests.get(url)
    weather_data = [{
        f'datetime': datetime.fromtimestamp(x['dt'], tz=pytz.utc),
        f'temp_{key}': x['main']['temp'],
        f'pressure_{key}': x['main']['pressure'],
        f'humidity_{key}': x['main']['humidity'],
        f'wind_speed_{key}': x['wind']['speed'],
        f'clouds_all_{key}': x['clouds']['all']
    } for x in response.json()['list']]
    weather_df = pd.DataFrame(weather_data)
    weather_dfs.append(weather_df.set_index('datetime'))
    
df = pd.concat(weather_dfs, axis=1)

forecast_dp = 48*28
date_index = pd.date_range(start=last_demand['local_datetime'], periods=forecast_dp+1, freq='30T')
df_reindexed = df.reindex(date_index)
df_interpolated = df_reindexed.interpolate(method='time', fill_value="extrapolate", limit_direction="both")
df_interpolated

,temp_london,pressure_london,humidity_london,wind_speed_london,clouds_all_london,temp_liverpool,pressure_liverpool,humidity_liverpool,wind_speed_liverpool,clouds_all_liverpool,temp_bath,pressure_bath,humidity_bath,wind_speed_bath,clouds_all_bath
2023-12-22 19:30:00+00:00,283.790000,1013.000000,76.000000,6.150000,40.000000,283.190000,1011.000000,89.000000,9.350000,75.000000,283.160000,1018.0,82.000000,6.760000,94.000000
2023-12-22 20:00:00+00:00,283.790000,1013.000000,76.000000,6.150000,40.000000,283.190000,1011.000000,89.000000,9.350000,75.000000,283.160000,1018.0,82.000000,6.760000,94.000000
2023-12-22 20:30:00+00:00,283.790000,1013.000000,76.000000,6.150000,40.000000,283.190000,1011.000000,89.000000,9.350000,75.000000,283.160000,1018.0,82.000000,6.760000,94.000000
2023-12-22 21:00:00+00:00,283.790000,1013.000000,76.000000,6.150000,40.000000,283.190000,1011.000000,89.000000,9.350000,75.000000,283.160000,1018.0,82.000000,6.760000,94.000000
2023-12-22 21:30:00+00:00,283.768333,1013.166667,75.666667,6.063333,43.166667,283.153333,1011.166667,88.333333,9.423333,76.333333,283.051667,1018.0,82.333333,6.718333,93.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-19 17:30:00+00:00,285.050000,996.000000,78.000000,11.080000,100.000000,281.200000,987.000000,88.000000,6.370000,100.000000,284.510000,992.0,91.000000,12.350000,100.000000
2024-01-19 18:00:00+00:00,285.050000,996.000000,78.000000,11.080000,100.000000,281.200000,987.000000,88.000000,6.370000,100.000000,284.510000,992.0,91.000000,12.350000,100.000000
2024-01-19 18:30:00+00:00,285.050000,996.000000,78.000000,11.080000,100.000000,281.200000,987.000000,88.000000,6.370000,100.000000,284.510000,992.0,91.000000,12.350000,100.000000
2024-01-19 19:00:00+00:00,285.050000,996.000000,78.000000,11.080000,100.000000,281.200000,987.000000,88.000000,6.370000,100.000000,284.510000,992.0,91.000000,12.350000,100.000000
